In [1]:
from air_hockey_challenge.framework.air_hockey_challenge_wrapper import AirHockeyChallengeWrapper

from air_hockey_challenge.environments.iiwas.env_hitting import AirHockeyHit
from air_hockey_agent.agent_builder import build_agent
import numpy as np

pybullet build time: May 20 2022 19:44:17


In [2]:
env = AirHockeyChallengeWrapper(env="7dof-hit", interpolation_order=3, debug=False)
# policy = build_agent(env.env_info)
policy = build_agent(env.env_info) 

In [3]:
env.env_info

{'table': {'length': 1.948, 'width': 1.038, 'goal_width': 0.25},
 'puck': {'radius': 0.03165},
 'mallet': {'radius': 0.04815},
 'n_agents': 2,
 'robot': {'n_joints': 7,
  'ee_desired_height': 0.1645,
  'joint_vel_limit': array([[-1.48352986, -1.48352986, -1.74532925, -1.30899694, -2.26892803,
          -2.35619449, -2.35619449],
         [ 1.48352986,  1.48352986,  1.74532925,  1.30899694,  2.26892803,
           2.35619449,  2.35619449]]),
  'joint_acc_limit': array([[-14.83529864, -14.83529864, -17.45329252, -13.08996939,
          -22.68928028, -23.5619449 , -23.5619449 ],
         [ 14.83529864,  14.83529864,  17.45329252,  13.08996939,
           22.68928028,  23.5619449 ,  23.5619449 ]]),
  'base_frame': [array([[ 1.  ,  0.  ,  0.  , -1.51],
          [ 0.  ,  1.  ,  0.  ,  0.  ],
          [ 0.  ,  0.  ,  1.  , -0.1 ],
          [ 0.  ,  0.  ,  0.  ,  1.  ]]),
   array([[-1.  ,  0.  ,  0.  ,  1.51],
          [ 0.  , -1.  ,  0.  ,  0.  ],
          [ 0.  ,  0.  ,  1.  , -0.1 ],


In [4]:
env.env_info['rl_info'].observation_space.low

array([   0.        ,   -1.        ,   -3.14159265,  -20.        ,
        -20.        , -100.        ,   -2.96706   ,   -2.0944    ,
         -2.96706   ,   -2.0944    ,   -2.96706   ,   -2.0944    ,
         -3.05433   ,   -1.48352986,   -1.48352986,   -1.74532925,
         -1.30899694,   -2.26892803,   -2.35619449,   -2.35619449,
          1.5       ,   -1.5       ,   -1.5       ])

In [651]:
state, done = env.reset(), False
env.render()


In [6]:
action = policy.draw_action(np.array(state))

# Perform action
next_state, reward, done, _ = env.step(action)
env.render()

In [7]:
from air_hockey_challenge.utils.kinematics import inverse_kinematics
import numpy as np


In [8]:
env.env_info['robot']['ee_desired_height']

0.1645

In [9]:
env.base_env.hit_range

array([[-0.7    , -0.2    ],
       [-0.39105,  0.39105]])

In [10]:
p1=np.array([-0.7,-0.39105,0]) + [1.51,0,0.1645]
p2=np.array([-0.7,0.39105,0]) + [1.51,0,0.1645]
p3=np.array([-0.2,-0.39105,0]) + [1.51,0,0.1645]
p4=np.array([-0.2,0.39105,0]) + [1.51,0,0.1645]

In [11]:
print(p1)
print(p2)
print(p3)
print(p4)

[ 0.81    -0.39105  0.1645 ]
[0.81    0.39105 0.1645 ]
[ 1.31    -0.39105  0.1645 ]
[1.31    0.39105 0.1645 ]


In [468]:
p1= np.array([0.65,-0.39,0.1654])
p2= np.array([0.65,0.39,0.1654])
p3= np.array([1.31,-0.39,0.1654])
p4= np.array([1.31,0.39,0.1654])

p5 = np.array([1.0,-0.39,0.1654])
p6 = np.array([1.0,0.39,0.1654]) 

In [652]:
angle_min = np.zeros((7)) + np.inf
angle_max = np.zeros((7)) - np.inf

vel_min = np.zeros((7)) + np.inf
vel_max = np.zeros((7)) - np.inf

In [794]:

des_pos = p4
# des_pos = [0.65,-0.39,0.1654]

state = next_state
action = np.zeros((2,7))
_,x = inverse_kinematics(policy.robot_model, policy.robot_data,des_pos)
des_v = np.array([0,0,0,0,0,0])
from air_hockey_challenge.utils.kinematics import jacobian
z = jacobian(policy.robot_model, policy.robot_data,policy.get_joint_pos(state))
z_=np.linalg.pinv(z)
v_ = des_v@z_.T
action = np.array([x,v_])
next_state, reward, done, _ = env.step(action)
env.render()
state = next_state
pos = state[6:13]
vel =  state[13:20]
angle_min[np.where(pos<angle_min)] = pos[np.where(pos<angle_min)]
angle_max[np.where(pos>angle_max)] = pos[np.where(pos>angle_max)]

vel_min[np.where(vel<vel_min)] = vel[np.where(vel<vel_min)]
vel_max[np.where(vel>vel_max)] = vel[np.where(vel>vel_max)]

print(angle_min)
print(angle_max)
print(vel_min)
print(vel_max)

[-0.40476327 -0.20591463 -0.25040716 -1.84715099 -0.28212878 -0.30128033
 -0.10303726]
[ 0.16055371  1.19900776  0.37962941 -0.92631058  0.3784197   1.19376235
 -0.01559517]
[-37.91439637  -0.6722146  -27.08600584  -2.03318772  -5.56511694
  -9.53569044  -5.03137651]
[26.11820054  2.07900234 38.40415028  2.22009052 22.74761576  7.13230693
  4.01165734]


In [795]:
print(angle_min)
print(angle_max)
print(vel_min)                                          #ignore vel limits
print(vel_max)

[-0.40476327 -0.20591463 -0.25040716 -1.84715099 -0.28212878 -0.30128033
 -0.10303726]
[ 0.16055371  1.19900776  0.37962941 -0.92631058  0.3784197   1.19376235
 -0.01559517]
[-37.91439637  -0.6722146  -27.08600584  -2.03318772  -5.56511694
  -9.53569044  -5.03137651]
[26.11820054  2.07900234 38.40415028  2.22009052 22.74761576  7.13230693
  4.01165734]


In [800]:
env.env_info['robot']['joint_pos_limit'][1] - angle_max

array([2.80650629, 0.89539224, 2.58743059, 3.02071058, 2.5886403 ,
       0.90063765, 3.06992517])